In [52]:
# import the libraries
# hint: sometimes execution break becuase e.g. you accidentally press something in the opened browser or connection issues or smth similar:
# normally restring the kernal and rerunning a particular period where it broke solves the issue
# normally I left this runnning overnight, it takes several hours to scrape 2007 - 2023

from bs4 import BeautifulSoup
from selenium import webdriver
from itertools import combinations
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import os

In [61]:
# Step 1: Download the ChromeDriver from the official website.
# Go to https://chromedriver.chromium.org/downloads and download the appropriate version for your operating system.

# Step 2: Adjust security settings (Mac specific).
# On Mac, you may encounter an error stating that the file cannot be opened due to security settings.
# To fix this, follow these steps:
#   - Open "System Preferences" from the Apple menu.
#   - Click on "Security & Privacy."
#   - In the "General" tab, you should see a message saying "chromedriver" was blocked from opening.
#   - Click the "Open Anyway" button next to the message.
#   - You might be prompted to enter your username and password.
#   - After allowing the file to open, continue with the steps below.

# Step 3: Set the path to the ChromeDriver executable.
# Replace the PATH variable below with the path to the ChromeDriver executable on your machine.
# Make sure to include the full path, including the folder and filename of the ChromeDriver.


PATH = r"/Users/juliaprozorova/Downloads/chromedriver_mac64-3/chromedriver"  # Replace with your own path

# Step 4: Create a WebDriver instance using ChromeDriver.
# The following code initializes the ChromeDriver with the provided path.
# It will allow you to automate Google Chrome using Selenium.

driver = webdriver.Chrome(PATH)


In [54]:
# define a function which is later used in scraping

def tick_courses (table, choice, a = None, b=None):
     """
     Tick Courses Function:
     Selects specific courses on a web page using Selenium WebDriver.

     Parameters:
     - table (WebElement): The table or container element containing the course options.
     - choice (str): The choice of courses to select. If "Faculty of Humanities and Social Sciences",
                            additional logic will be applied due to different page rendering.
     - a (str): Optional parameter. Text to exclude from selection if "Faculty of Humanities and Social Sciences" is chosen.
     - b (str): Optional parameter. Text to exclude from selection if "Faculty of Humanities and Social Sciences" is chosen.
     """
     if choice == "Faculty of Humanities and Social Sciences":
            found1 = False
            found2 = False
            found = False

            for el in table.find_elements(By.TAG_NAME,"li"):
                for e in reversed(el.find_elements(By.TAG_NAME,"span")):
                    
                    if e.get_attribute("class") == "fancytree-title":
                          if e.text == "Faculty of Humanities and Social Sciences":
                            found1 = True
                            found2 = True

                    elif found1 and e.get_attribute("class") == "fancytree-expander":
                        webdriver.ActionChains(driver).move_to_element(e).click().perform()
                        found1 = False
                        
   
                    elif found2 and e.get_attribute("class") == "fancytree-checkbox":
                        webdriver.ActionChains(driver).move_to_element(e).click().perform()
                        found2 = False
                                  
                                            
            done = 0
            for el in table.find_elements(By.TAG_NAME,"li"):
                for e in reversed(el.find_elements(By.TAG_NAME,"span")):
                    if e.get_attribute("class") == "fancytree-title":
         
                        if a in e.text  or b in e.text : 

                            found = True
                           

                    elif done < 2 and found and e.get_attribute("class") == "fancytree-checkbox":
                        webdriver.ActionChains(driver).move_to_element(e).click().perform()
                        found = False
                        done+=1
                        
     else:
            found = False
            for el in table.find_elements(By.TAG_NAME,"li"):
                for e in reversed(el.find_elements(By.TAG_NAME,"span")):
                    if e.get_attribute("class") == "fancytree-title":
                        if e.text == choice:
                            found = True
                    elif found and e.get_attribute("class") == "fancytree-checkbox":
                        webdriver.ActionChains(driver).move_to_element(e).click().perform()
                        found = False
                        
     apply = driver.find_elements_by_xpath("//*[contains(text(), 'Apply')]")
     webdriver.ActionChains(driver).move_to_element(apply[0]).click().perform()
     
     # unfreeze
     driver.find_element_by_xpath("//body").click() # apply =  driver.find_element(By.CLASS_NAME, "col-lg-12")
     #webdriver.ActionChains(driver).move_to_element(apply).click().perform()
    
     #make sure it was pressed properly
     apply = driver.find_elements_by_xpath("//*[contains(text(), 'Show results')]")
     webdriver.ActionChains(driver).move_to_element(apply[0]).click().perform()
     time.sleep(20) 
                          

In [55]:
def scrape_list(m, a = None, b = None, additional_name = ""):
            """
            Scrape List Function:
            Scrapes a list of courses from a web page using Selenium WebDriver and saves the results to a CSV file.

            Parameters:
                - m (str): The choice of courses to select.
                - a (str): Optional parameter. Text to exclude from selection if "Faculty of Humanities and Social Sciences" is chosen.
                - b (str): Optional parameter. Text to exclude from selection if "Faculty of Humanities and Social Sciences" is chosen.
                - additional_name (str): Optional parameter. The name to be included in the resulting CSV filename.

            Returns:
                None. The function saves the scraped course data to a CSV file.

            """
    # first open, unfreeze, then press reset
            el = driver.find_elements_by_xpath("//*[contains(text(), 'Program structure')]")
            webdriver.ActionChains(driver).move_to_element(el[1]).click().perform()
            time.sleep(1)
            #p = driver.find_element(By.TAG_NAME,"p"); 
            #webdriver.ActionChains(driver).move_to_element(p).click().perform()
            apply = driver.find_elements_by_xpath("//*[contains(text(), 'Reset')]")
            webdriver.ActionChains(driver).move_to_element(apply[2]).click().perform()

            #open from the beginning
            x = "https://vorlesungsverzeichnis.unibas.ch/en/investigation?periode=2021005&search=1"
            x = x.replace("2021005", semester)
            driver.get(x)
            # adjust settings
            el = driver.find_elements_by_xpath("//*[contains(text(), 'Program structure')]")
            webdriver.ActionChains(driver).move_to_element(el[1]).click().perform()
            time.sleep(1)
            table = driver.find_element(By.XPATH, "//ul[@class='ui-fancytree fancytree-container fancytree-plain']")


            tick_courses(table, m, a, b)


            # construct database
            columns = ["index", "name", "link"]
            df = pd.DataFrame(columns =columns)      

            try:
                table = driver.find_element(By.XPATH, "//table[@class='table table-striped table-bordered table-hover dataTable no-footer']")
                print(table)
                while True:
                    try: 
                        rows = table.find_elements(By.TAG_NAME, "tr")
                        id_ = rows[1].find_elements(By.TAG_NAME, "td")[1].text
                        if id_ in set(df["index"]):
                                    print("Duplicate entry encountered. Everything is processed. Exiting loop.")
                                    break

                        k = 0
                        for row in rows:
                            if k >0:   
                                    id_ = row.find_elements(By.TAG_NAME, "td")[1].text
                                    name = row.find_elements(By.TAG_NAME, "td")[3].text
                                    link = row.find_elements(By.TAG_NAME, "td")[3].find_elements(By.TAG_NAME, "a")[0].get_attribute('href')
                                    row = [id_, name, link]
                                    df.loc[len(df.index)] = row
                            k +=1
                        next_page = driver.find_elements_by_xpath("//*[contains(text(), 'Next')]")
                        webdriver.ActionChains(driver).move_to_element(next_page[0]).click().perform()       

                    except Exception as e: 
                        print("Error occurred while scraping the course list. Exiting loop.", x, e)
                        break
                # save the result, adjust the path where you want to save the resulting csv
                df.to_csv('test/courses_%s_%s_%s.csv' %(semester, m, additional_name),  encoding='utf-8', index=False) #fixme 
                print("Scraping complete. Data saved to courses_%s_%s_%s.csv" %(semester, m, additional_name))
            except Exception as e: 
                    print("An exception occurred during scraping:", e)



In [58]:
# This script performs the first step of the scraping process to retrieve the "index," "name," and "link" for each course from the University course catalog. 
#It iterates over different semesters and faculties, navigates to the respective pages, 
#and extracts the course information. The script utilizes Selenium and BeautifulSoup to interact with the web pages and parse the HTML content. 
#The extracted data is stored in CSV files for further processing in the second step. 
#For the "Faculty of Humanities and Social Sciences," the script retrieves course information for different combinations of "Bachelor," "Master," and "Doktorat" programs.
#This is because simulatanous processsing of all results in error due to large number of courses

for year in range (2007, 2008):
    for period in range (4, 5):
        semester = ind = str(year)+("00")+str(period)

        
        # open a page once for each semester
        x = "https://vorlesungsverzeichnis.unibas.ch/en/investigation?periode=2021005&search=1"
        x = x.replace("2021005", semester)
        driver.get(x)

        # constructing the list of all the faculties
        mylist = []
        el = driver.find_elements_by_xpath("//*[contains(text(), 'Program structure')]")
        webdriver.ActionChains(driver).move_to_element(el[1]).click().perform()
        time.sleep(1)
        table = driver.find_element(By.XPATH, "//ul[@class='ui-fancytree fancytree-container fancytree-plain']")
        for el in table.find_elements(By.TAG_NAME,"li"):
                    for e in reversed(el.find_elements(By.TAG_NAME,"span")):
                        if e.get_attribute("class") == "fancytree-title":
                            mylist.append(e.text)

        x = "https://vorlesungsverzeichnis.unibas.ch/en/investigation?periode=2021005&search=1"
        x = x.replace("2021005", semester)
        driver.get(x)  

        for m in mylist:
            if m == "Faculty of Humanities and Social Sciences":
                for (a, b) in combinations(["Bachelor", "Master", "Doktorat"], 2):
                    additional_name = (set(["Bachelor", "Master", "Doktorat"]) - set([a, b])).pop()
                    scrape_list(m, a, b, additional_name)
            else:
                scrape_list(m)

            
            

<selenium.webdriver.remote.webelement.WebElement (session="79bcb79a92184577b4f6d1a76f5d13c5", element="16D7A8CBEA9EAF9985D16BDD4C441A4E_element_35130")>
Duplicate entry encountered. Everything is processed. Exiting loop.
Scraping complete. Data saved to courses_2007004_University of Basel_.csv
<selenium.webdriver.remote.webelement.WebElement (session="79bcb79a92184577b4f6d1a76f5d13c5", element="BA7BF559A5681973FF6E2E78036D6F7F_element_36710")>
Error occurred while scraping the course list. Exiting loop. https://vorlesungsverzeichnis.unibas.ch/en/investigation?periode=2007004&search=1 list index out of range
Scraping complete. Data saved to courses_2007004_Faculty of Theology_.csv
An exception occurred during scraping: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//table[@class='table table-striped table-bordered table-hover dataTable no-footer']"}
  (Session info: chrome=114.0.5735.133)

<selenium.webdriver.remote.webelement.WebElement (session="79b

In [62]:
# This script performs the second step of the scraping process 
#to retrieve detailed data for the courses obtained in the first step. 
#It reads CSV files containing the course information, scrapes the individual course pages to extract additional details, 
#and saves the combined data into the same CSV files. The script uses BeautifulSoup to parse the HTML content of each course page and retrieves specific elements. 
#The extracted data is then structured into a DataFrame and concatenated with the existing course data. 
#The resulting DataFrame is saved back to the CSV files. 
#The script also handles exceptions to ensure smooth processing of multiple files.

# adjust the path to where you saved the results from the first step
dirr = os.getcwd()+"/test" #fixme

# Iterate over each file in the directory
for filename in os.listdir(dirr):
                try:
                    # Read the CSV file containing the scraped course data
                    df = pd.read_csv('test/'+filename ) #fixme
                    print(filename, len(df)) # manual step: cross check with the website that the number of courses is correct and everyhting got sraped
                    # Get the links to each course page
                    links = list(df["link"])
                    # Initialize the final data DataFrame
                    final_data = None
                    # Iterate over each course link
                    for link in links:
                        #link =  link.replace("en/semester-planning", "de/semester-planung")
                        driver.get(link)
                        soup = BeautifulSoup(driver.page_source,'html.parser')
                        data = pd.DataFrame(columns =["course_name", link])
                        # Find the course information using BeautifulSoup
                        s = soup.findAll("strong")
                        for el in s:
                            new = soup.find("td", text=el.text).find_next_sibling("td").text
                            row = [el.text, new]
                            data.loc[len(data)]=row
                        data.set_index('course_name',inplace=True)
                        data = data.transpose()
                        for el in soup.findAll("li"):
                            if " CP" in el.text:
                                data["full_name"] = el.text
                        # Concatenate the course data to the final_data DataFrame
                        if final_data is not None:
                            final_data = pd.concat([final_data, data], ignore_index=False)
                        else:
                            final_data = data
                     # Save the final_data DataFrame to a CSV file
                    final_data.to_csv('test/scraped/'+filename,  encoding='utf-8', index=False)#fixme
                except Exception as e:
                    print("An exception occurred while processing file:", filename, e)

courses_2007004_Teacher training program_.csv 65
courses_2007004_Faculty of Theology_.csv 0
An exception occurred while processing file: courses_2007004_Faculty of Theology_.csv 'NoneType' object has no attribute 'to_csv'
courses_2007004_Faculty of Psychology_.csv 42
courses_2007004_Faculty of Humanities and Social Sciences_Master.csv 385
An exception occurred while processing file: scraped [Errno 21] Is a directory: 'test/scraped'
courses_2007004_Faculty of Science_.csv 556
courses_2007004_University of Basel_.csv 128
courses_2007004_Faculty of Humanities and Social Sciences_Doktorat.csv 40
An exception occurred while processing file: .ipynb_checkpoints [Errno 21] Is a directory: 'test/.ipynb_checkpoints'
courses_2007004_Faculty of Business and Economics_.csv 45
courses_2007004_Faculty of Medicine_.csv 149
courses_2007004_Faculty of Humanities and Social Sciences_Bachelor.csv 695
